In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
REST = "nyc_restaurant_inspections.csv"

In [3]:
rests = sc.textFile(REST, use_unicode=True).cache()

In [4]:
list(enumerate(rests.first().split(',')))

[(0, 'CAMIS'),
 (1, 'DBA'),
 (2, 'BORO'),
 (3, 'BUILDING'),
 (4, 'STREET'),
 (5, 'ZIPCODE'),
 (6, 'PHONE'),
 (7, 'CUISINE DESCRIPTION'),
 (8, 'INSPECTION DATE'),
 (9, 'ACTION'),
 (10, 'VIOLATION CODE'),
 (11, 'VIOLATION DESCRIPTION'),
 (12, 'CRITICAL FLAG'),
 (13, 'SCORE'),
 (14, 'GRADE'),
 (15, 'GRADE DATE'),
 (16, 'RECORD DATE'),
 (17, 'INSPECTION TYPE')]

In [5]:
rests.take(2)

['CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE',
 '50015075,NEW YORK FRIED CHICKEN,BROOKLYN,102,SARATOGA AVE,11233,7185522345,Chicken,04/28/2015,Violations were cited in the following area(s).,02G,Cold food item held above 41Âº F (smoked fish and reduced oxygen packaged foods above 38 ÂºF) except during necessary preparation.,Critical,16,B,04/28/2015,03/01/2018,Cycle Inspection / Re-inspection']

In [6]:
def extractRestaurants(partId, records):
    import csv
    if partId==0:
        next(records)
    reader = csv.reader(records)
    for row in reader:
        (camis, desc) = (row[0], row[7])
        yield (camis, desc)

In [7]:
nyRests = rests.mapPartitionsWithIndex(extractRestaurants)

In [8]:
nyRests.take(5)

[('50015075', 'Chicken'),
 ('40364529', 'Jewish/Kosher'),
 ('41580756', 'Caribbean'),
 ('50001255', 'Italian'),
 ('50016437', 'Korean')]

In [9]:
restList = nyRests \
.map(lambda x: (x, 1)) \
.reduceByKey(lambda x,y: 1) \
.keys().map(lambda x: (x[1],1)) \
.reduceByKey(lambda x,y: x+y) \
.top(25, key=lambda x: x[1])

In [10]:
restList

[('American', 6002),
 ('Chinese', 2399),
 ('CafÃ©/Coffee/Tea', 1629),
 ('Other', 1296),
 ('Pizza', 1186),
 ('Italian', 1016),
 ('Mexican', 877),
 ('Japanese', 859),
 ('Latin (Cuban, Dominican, Puerto Rican, South & Central American)', 840),
 ('Bakery', 733),
 ('Caribbean', 671),
 ('Spanish', 644),
 ('Donuts', 537),
 ('Pizza/Italian', 483),
 ('Chicken', 456),
 ('Sandwiches', 406),
 ('Juice, Smoothies, Fruit Salads', 382),
 ('Hamburgers', 378),
 ('Asian', 371),
 ('Ice Cream, Gelato, Yogurt, Ices', 339),
 ('Indian', 332),
 ('Jewish/Kosher', 327),
 ('French', 319),
 ('Delicatessen', 294),
 ('Thai', 286)]